In [ ]:
import numpy as np
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
from pathlib import Path
import velocity_estimation.two_dim_velocity_estimates as tdve
import velocity_estimation.utils as u
import velocity_estimation.time_delay_estimation as td
from fppanalysis.running_moments import window_radius, run_norm_ds, run_mean_ds
import cosmoplots
import tde_functions
from w7x_array import 

#### Create xarray dataset from hdf5 file

In [ ]:
shot = 230323039

filename = '/Users/ahe104/Box/W7X_data/W7X_gpi_h5_files/'+str(shot).strip()+'.h5'
rad_pol_filename = np.load('/Users/ahe104/Box/W7X_data/rz_arrays/rz_arrs.npz')

z_arr, r_arr, pol_arr, rad_arr = rad_pol_positions(rad_pol_filename)
r_z_coord = False
if r_z_coord:
    ds = create_xarray_from_hdf5(filename, r_arr, z_arr)
else:
    ds = create_xarray_from_hdf5(filename, rad_arr, pol_arr)

#### Slice dataset

In [ ]:
t_start = 7.1
t_end = 7.15
sliced_ds = ds.sel(time=slice(t_start, t_end))
frames = sliced_ds['frames']

#### Detrend data

In [ ]:
cut_off_freq = 1e3
#cut_off_freq = 5e3
radius = window_radius(cut_off_freq, ds.time.values)
ds = run_norm_ds(sliced_ds, radius=radius)

#### Estimation options for velocity estimation

In [ ]:
eo = tdve.EstimationOptions()
eo.use_3point_method = True
eo.cc_options.running_mean = True
eo.cc_options.minimum_cc_value = 0
eo.neighbour_options = tdve.NeighbourOptions(ccf_min_lag=0, max_separation=1, min_separation=1)

#### Generate velocities

In [ ]:
movie_data = tdve.estimate_velocity_field(u.CModImagingDataInterface(ds), eo)
vx = movie_data.get_vx()
vy = movie_data.get_vy()
confidences = movie_data.get_confidences()
R = movie_data.get_R()
Z = movie_data.get_Z()

#### Plot velocity field

In [ ]:
axes_size = cosmoplots.set_rcparams_dynamo(plt.rcParams, num_cols=1, ls="thin")
plt.rcParams["mathtext.fontset"] = "custom"

fig = plt.figure()
ax = fig.add_axes(axes_size)

ax.scatter(R, Z, marker=".", color='midnightblue', s=0.5)

default_vmin = 0
default_vmax = 1
norm = mpl.colors.Normalize(vmin=default_vmin, vmax=default_vmax)

qiv = ax.quiver(
    R,
    Z,
    vx,
    vy,
    confidences,
    scale=300000,
    scale_units="xy",
    angles="xy",
    norm=norm,
    width=0.01
)

qk = ax.quiverkey(
    qiv, 0.5, 0.06, 300000, r"$3000$ m/s", labelpos="E", coordinates="axes"
)
qk = ax.quiverkey(
    qiv, 0.5, 0.12, 200000, r"$2000$ m/s", labelpos="E", coordinates="axes"
)
qk = ax.quiverkey(
    qiv, 0.5, 0.18, 100000, r"$1000$ m/s", labelpos="E", coordinates="axes"
)

cbar = fig.colorbar(qiv, format="%.1f")
cbar.ax.set_ylabel(r"max $\hat{R}_{\tilde{{\Phi}}}$", rotation=270, labelpad=13)
if r_z_coord:
    ax.set_xlabel(r"$R$ / cm")
    ax.set_ylabel(r"$Z$ / cm")
else:
    ax.set_xlabel(r"$R - R (15.0)$ / cm")
    ax.set_ylabel(r"$Z - Z (15.0)$ / cm") 
ax.set_aspect("equal")

if use_2d_estimation:
    if r_z_coord:
        ax.set_ylim(min(Z[:,0]) - 1.95, max(Z[:,0] + 0.6))
        ax.set_xlim([min(R[0]) - 3, max(R[0]) + 0.5])
        plt.xticks(np.arange(round(min(R[0])-2), round(max(R[0]))+1, 2))
    else:
        ax.set_ylim(min(Z[:,0]) - 0.5, max(Z[:,0] + 0.5))
        ax.set_xlim([min(R[0]) - 0.5, max(R[0]) + 0.5])
        plt.xticks(np.arange(round(min(R[0])), round(max(R[0]))+1, 1))
else:
    if r_z_coord:
        ax.set_ylim(min(Z[:,0]) - 2, max(Z[:,0] + 1))
        ax.set_xlim([min(R[0]) - 1, max(R[0]) + 1])
        plt.xticks(np.arange(round(min(R[0])-2), round(max(R[0]))+1, 2))
    else:
        ax.set_ylim(min(Z[:,0]) - 3, max(Z[:,0] + 3))
        ax.set_xlim([min(R[0]) - 3, max(R[0]) + 3])
        plt.xticks(np.arange(round(min(R[0])-2), round(max(R[0]))+3, 2))
# plt.yticks(np.arange(round(min(Z_coordinates[0])-3), round(max(R_coordinates[0]))+3, 1))
# ax.set_xlim([min(R) - 0.5, max(R) + 0.5])
# ax.set_ylim([min(Z) - 0.5, max(Z) + 0.5])

title_prefix = f"2 point" if not use_2d_estimation else f"3 point"
title =  rf"Shot {shot} " "\n"  f"{title_prefix}"
ax.set_title(title, size=8)